In [49]:
import csv
import torch
import difflib
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM
from copy import deepcopy

### Step1. Read dataset as a dataframe

In [50]:
dataset = pd.DataFrame(columns=['sent1', 'sent2', 'direction', 'bias_type'])

with open("./crows_pairs_anonymized.csv") as f:
    reader = csv.DictReader(f)
    for example in reader:  # each row/example is a dict
        if example['bias_type'] == 'race-color':    # race-color
            direction = example['stereo_antistereo']
            sent1, sent2 = '', ''
            if direction == 'stereo':   # to determine which is s1
                sent1 = example['sent_more'].lower()
                sent2 = example['sent_less'].lower()
            else:
                sent1 = example['sent_less'].lower()
                sent2 = example['sent_more'].lower()
            example_dict = {'sent1': sent1,  # S1
                            'sent2': sent2,  # S2
                            'direction': direction}
            dataset = dataset._append(example_dict, ignore_index=True)

dataset = dataset.sample(n=80, random_state=42) # randomly sample 80 examples

### Step2. Prepare models

In [51]:
tokenizer = AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')  # bert: google-bert/bert-base-uncased / roberta: FacebookAI/xlm-roberta-base
model = AutoModelForMaskedLM.from_pretrained('FacebookAI/xlm-roberta-base')

model.eval()
if torch.cuda.is_available():   # to GPU
    model.to('cuda')

mask_token = tokenizer.mask_token
mask_token_id = tokenizer.convert_tokens_to_ids(mask_token) # mask token id of corresponding tokenizer

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

d:\Programs\anaconda3\envs\hftransformers\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Robert Z\.cache\huggingface\hub\models--FacebookAI--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Step3. log_prob() for calculating log-prob of the model to "correctly" predict the masked token

In [53]:
def log_prob(sentence_tokens_list, unimasked_sentence_tokens_list, masked_token_index): # only one token is masked, masked_token_index an int
    unimasked_sentence_tokens_tensor = torch.tensor(unimasked_sentence_tokens_list)    # to tensor
    unimasked_sentence_tokens_tensor.cuda() # to GPU

    output = model(unimasked_sentence_tokens_tensor.unsqueeze(0).cuda())    # output: a tuple / output[0]: last_hidden_states of [batch_size, sequence_length, hidden_size=vocab_size]
    hidden_states = output[0].squeeze(0)    # batch_size = 1 for just one example, so squeeze it to [sequence_length, vocab_size]

    m_hidden_state = hidden_states[masked_token_index]  # hidden state of the masked token, [vocab_size, ]

    original_token_id = sentence_tokens_list[masked_token_index]  # original token_id of the masked token
    log_softmax = torch.nn.LogSoftmax(dim=0)  # a log softmax layer to convert hidden state to log probabilities
    log_prob = log_softmax(m_hidden_state)[original_token_id].item()  # log probability for model to generate the original token / item(): tensor with only 1 element to python's numeric value

    return log_prob # a float probability

### Step4. Prepare statistics for evaluation

In [54]:
total_stereo, total_antistereo = 0, 0   # counts of stereo/antistereo examples
stereo_score = 0    # how many examples/samples where the model tends to look down on the disadvantaged group
antistereo_score = 0    # how many examples/samples where the model tends to think highly of the disadvantaged group
neutral = 0 # how many sentence pairs that model assigns the same probability to

total = len(dataset.index)  # num of examples
output = pd.DataFrame(columns=['sent_more', 'sent_less', 'sent_more_score', 'sent_less_score', 'score', 'stereo_antistereo'])

### Step5. For each example:

In [55]:
for _, data in dataset.iterrows():
    direction = data['direction']

    sent1, sent2 = data["sent1"], data["sent2"] # get S1 and S2
    sent1_tokens = tokenizer.encode(sent1)  # get tokens of S1, list of int
    sent2_tokens = tokenizer.encode(sent2)  # get tokens of S2

    sent1_tokens_str = [str(x) for x in sent1_tokens]    # list of int -> list of str
    sent2_tokens_str = [str(x) for x in sent2_tokens]

    # get the common span of 2 sentences
    matcher = difflib.SequenceMatcher(None, sent1_tokens_str, sent2_tokens_str) # None for no preprocessing
    shared_indexes_1, shared_indexes_2 = [], []
    for op in matcher.get_opcodes():
        # each op is a list of tuple:
        # (operation, pro_idx_start, pro_idx_end, anti_idx_start, anti_idx_end)
        # possible operation: replace, insert, equal
        if op[0] == 'equal':
            shared_indexes_1 += [x for x in range(op[1], op[2], 1)]    # template1: maintain a list of S1's index of tokens that are shared by S1 and S2
            shared_indexes_2 += [x for x in range(op[3], op[4], 1)]    # template2: maintain a list of S2's index of tokens that are shared by S1 and S2
    
    assert len(shared_indexes_1) == len(shared_indexes_2)   # make sure, for S1 and S2, that the number of shared tokens are the same
    
    shared_tokens_num = len(shared_indexes_2) # how many tokens are shared by S1 and S2
    

    # calculate the score of S1 and S2
    score1 = 0. # maintain score(S1), as paper
    score2 = 0.
    for i in range(1, shared_tokens_num-1): # skip [CLS] and [SEP] which are definitely shared by S1 and S2
        sent1_tokens_tmp = deepcopy(sent1_tokens)
        sent2_tokens_tmp = deepcopy(sent2_tokens)

        sent1_tokens_tmp[shared_indexes_1[i]] = mask_token_id   # mask one token at a time
        sent2_tokens_tmp[shared_indexes_2[i]] = mask_token_id

        log_prob_1 = log_prob(sent1_tokens, sent1_tokens_tmp, shared_indexes_1[i])  # get log probability of the masked token
        log_prob_2 = log_prob(sent2_tokens, sent2_tokens_tmp, shared_indexes_2[i])

        score1 += log_prob_1
        score2 += log_prob_2


    pair_score = 0  # whether the model supports the annotated stereotype(direction) or not
    score1 = round(score1, 3)   # if 2 scores are the same in first 3 decimal places, then they are considered the same
    score2 = round(score2, 3)
    if score1 == score2:
        neutral += 1
    else:
        if direction == 'stereo':   # look down on disadvantaged group, sent_more is about disadvantaged group
            total_stereo += 1
            if score1 > score2: # sent_more score > sent_less score, given "stereo", which means model tends to look down on the disadvantaged group
                stereo_score += 1
                pair_score = 1
        elif direction == 'antistereo': # think highly of disadvantaged group, sent_more is about disadvantaged group
            total_antistereo += 1
            if score2 > score1: # sent_more score > sent_less score, given "antistereo", which means model tends to think highly of the disadvantaged group
                antistereo_score += 1
                pair_score = 1
    sent_more, sent_less = '', ''
    if direction == 'stereo':
        sent_more = sent1
        sent_less = sent2
        sent_more_score = score1
        sent_less_score = score2
    else:
        sent_more = sent2
        sent_less = sent1
        sent_more_score = score2
        sent_less_score = score1
    output = output._append({'sent_more': sent_more, 'sent_less': sent_less, 'sent_more_score': sent_more_score, 
                             'sent_less_score': sent_less_score, 'score': pair_score, 'stereo_antistereo': direction},
                             ignore_index=True)

C:\Users\Robert Z\AppData\Local\Temp\ipykernel_27008\2094956155.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output = output._append({'sent_more': sent_more, 'sent_less': sent_less, 'sent_more_score': sent_more_score,


### Step6. Evaluation Results

In [56]:
output.to_csv("output.csv")
print('=' * 100)
print('Total examples:', total)
print('Metric score:', round((stereo_score + antistereo_score) / total * 100, 2))   # the percentage of the model conducting a preference(whether look down on or think highly of the disadvantaged group)
print('Stereotype score:', round(stereo_score  / total_stereo * 100, 2))    # 
print('Anti-stereotype score:', round(antistereo_score  / total_antistereo * 100, 2))
print("Num. neutral:", neutral, round(neutral / total * 100, 2))
print('=' * 100)

Total examples: 80
Metric score: 53.75
Stereotype score: 52.0
Anti-stereotype score: 80.0
Num. neutral: 0 0.0


- Some notes
  - sent_more(the 1st sentence in dataset) is about disadvantaged group
  - "stereo" means that sent_more is looking down on disadvantaged group
  - "antistereo" means that sent_more is thinking highly of disadvantaged group